In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
#!pip install pyjson
#!pip install pandas
#!pip install geopandas
#!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
#!pip install -U pandas-profiling

In [3]:
%cd /gdrive/My Drive/Data Quality

/gdrive/My Drive/Data Quality


In [4]:
import random
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
import tensorflow as tf

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

tfk = tf.keras
tfkl = tf.keras.layers

In [5]:
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [6]:
def check_datatypes(df):
    for col in df.columns:
        if (df[col].dtype == "bool"):
            df[col] = df[col].astype('string')
            df[col] = df[col].astype('object')
    return  df

def injection(df_pandas, seed, name, name_class):

    np.random.seed(seed)

    #%%

    df_list = []

    perc = [0.50, 0.40, 0.30, 0.20, 0.10]
    for p in perc:
        df_dirt = df_pandas.copy()
        comp = [p,1-p]
        df_dirt = check_datatypes(df_dirt)

        for col in df_dirt.columns:

            if col!=name_class:

                rand = np.random.choice([True, False], size=df_dirt.shape[0], p=comp)

                df_dirt.loc[rand == True,col]=np.nan

        df_list.append(df_dirt)
        print("saved {}-completeness{}%".format(name, round((1-p)*100)))
    return df_list


In [7]:
adult = pd.read_csv("adult.csv")
adult

,age,workclass,fnlwgt,education,education-num,maritial-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,42,Self-emp-not-inc,99185,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
1,55,Private,163083,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,>50K
2,49,Private,340755,Masters,14,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States,>50K
3,31,Private,208881,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
4,19,Private,271446,Some-college,10,Never-married,Other-service,Own-child,White,Female,0,0,25,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3012,37,Local-gov,160910,Bachelors,13,Married-civ-spouse,Adm-clerical,Husband,White,Male,7298,0,40,United-States,>50K
3013,30,Private,488720,9th,5,Married-civ-spouse,Handlers-cleaners,Other-relative,White,Male,0,0,40,Mexico,<=50K
3014,37,Private,138940,11th,7,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
3015,34,Private,225548,Bachelors,13,Married-civ-spouse,Protective-serv,Husband,Black,Male,0,0,30,United-States,<=50K


In [8]:
#profile = ProfileReport(adult, title="Report")
#profile.to_file("df_report.html")

In [9]:
df_list= injection(adult, seed, "adult", "income")
df_Nan = pd.DataFrame(df_list[4], columns=adult.columns)
#d = {'<=50K': 0, '>50K': 1}
#df_Nan["income"] = df_Nan["income"].map(d)
df_Nan

saved adult-completeness50%
saved adult-completeness60%
saved adult-completeness70%
saved adult-completeness80%
saved adult-completeness90%


,age,workclass,fnlwgt,education,education-num,maritial-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,42.0,Self-emp-not-inc,99185.0,NaN,10.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
1,55.0,Private,163083.0,Masters,14.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
2,49.0,Private,340755.0,Masters,14.0,NaN,Sales,Husband,White,Male,0.0,0.0,40.0,NaN,>50K
3,31.0,NaN,208881.0,Some-college,10.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0.0,0.0,40.0,United-States,<=50K
4,19.0,Private,271446.0,Some-college,10.0,Never-married,Other-service,Own-child,White,Female,NaN,0.0,NaN,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3012,37.0,Local-gov,160910.0,Bachelors,13.0,Married-civ-spouse,Adm-clerical,Husband,White,Male,7298.0,0.0,40.0,United-States,>50K
3013,30.0,NaN,488720.0,9th,5.0,Married-civ-spouse,Handlers-cleaners,Other-relative,White,Male,0.0,0.0,40.0,Mexico,<=50K
3014,NaN,Private,138940.0,11th,7.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0.0,0.0,NaN,United-States,<=50K
3015,34.0,Private,225548.0,Bachelors,13.0,Married-civ-spouse,Protective-serv,Husband,Black,Male,NaN,NaN,NaN,United-States,<=50K


In [10]:
#profile = ProfileReport(df_Nan, title="Report")
#profile.to_file("df_report_Nan.html")

# Standard Imputation techniques

In [11]:
df_standard = df_Nan.copy()

In [12]:
df_standard["age"] = df_standard["age"].fillna(df_standard["age"].mean())
df_standard["workclass"] = df_standard["workclass"].fillna(method = "bfill")
df_standard["fnlwgt"] = df_standard["fnlwgt"].fillna(random.randint(adult["fnlwgt"].min(), adult["fnlwgt"].max()))
df_standard["education"] = df_standard["education"].fillna(method = "bfill")
df_standard["education-num"] = df_standard["education-num"].fillna(df_standard["education-num"].mean())
df_standard["maritial-status"] = df_standard["maritial-status"].fillna(method = "bfill")
df_standard["occupation"] = df_standard["occupation"].fillna(method = "bfill")
df_standard["relationship"] = df_standard["relationship"].fillna(method = "bfill")
df_standard["race"] = df_standard["race"].fillna(method = "bfill")
df_standard["sex"] = df_standard["sex"].fillna(method = "bfill")
df_standard["capital-gain"] = df_standard["capital-gain"].fillna(0)
df_standard["capital-loss"] = df_standard["capital-loss"].fillna(0)
df_standard["hours-per-week"] = df_standard["hours-per-week"].fillna(df_standard["hours-per-week"].mean())
df_standard["native-country"] = df_standard["native-country"].fillna("United-States")

In [13]:
total = 0
right = 0

for col in adult.columns:
  for row in range(0, adult.shape[0]):
    if col != "income":
      if pd.isna(df_Nan[col][row]):
        total = total + 1
        if adult[col][row] == df_standard[col][row]:
          right = right + 1

total, right
accuracy_standard = right/total
print(total)
print(accuracy_standard)

4335
0.4


In [14]:
target = "income"
features = df_standard.columns[df_standard.columns != target]

x = df_standard[features]
x = pd.get_dummies(x)
y = df_standard[target]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=seed, shuffle=True, stratify=y)

# Logistic Regression

In [15]:
logistic = LogisticRegression(random_state=seed, solver="liblinear")
logistic.fit(x_train,y_train)
yp = logistic.predict(x_test)
logistic.classes_

array(['<=50K', '>50K'], dtype=object)

In [16]:
accuracy = accuracy_score(y_test, yp)
precision = precision_score(y_test, yp, pos_label = ">50K")
recall = recall_score(y_test, yp, pos_label = ">50K")
f1 = f1_score(y_test, yp, pos_label = ">50K")
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

cm = confusion_matrix(y_test, yp)
plt.figure(figsize=(10,8))
sns.heatmap(cm, cmap='Blues', xticklabels=["<=50K", ">50K"], yticklabels=["<=50K", ">50K"])
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

Accuracy: 0.7748
Precision: 0.7222
Recall: 0.1711
F1: 0.2766


# KNN

In [17]:
knn = KNeighborsClassifier(n_neighbors=5, algorithm='kd_tree')
knn.fit(x_train,y_train)
y_predicted = knn.predict(x_test)
knn.classes_

array(['<=50K', '>50K'], dtype=object)

In [18]:
accuracy = accuracy_score(y_test, y_predicted)
precision = precision_score(y_test, y_predicted, pos_label = ">50K")
recall = recall_score(y_test, y_predicted, pos_label = ">50K")
f1 = f1_score(y_test, y_predicted, pos_label = ">50K")
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

cm = confusion_matrix(y_test, y_predicted)
plt.figure(figsize=(10,8))
sns.heatmap(cm, cmap='Blues', xticklabels=["<=50K", ">50K"], yticklabels=["<=50K", ">50K"])
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

Accuracy: 0.7401
Precision: 0.4545
Recall: 0.1645
F1: 0.2415


# AdaBoost Classifier

In [19]:
ada_boost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3), n_estimators=50)
ada_boost.fit(x_train,y_train)
y_predicted = ada_boost.predict(x_test)
ada_boost.classes_

array(['<=50K', '>50K'], dtype=object)

In [20]:
accuracy = accuracy_score(y_test, y_predicted)
precision = precision_score(y_test, y_predicted, pos_label = ">50K")
recall = recall_score(y_test, y_predicted, pos_label = ">50K")
f1 = f1_score(y_test, y_predicted, pos_label = ">50K")
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

cm = confusion_matrix(y_test, y_predicted)
plt.figure(figsize=(10,8))
sns.heatmap(cm, cmap='Blues', xticklabels=["<=50K", ">50K"], yticklabels=["<=50K", ">50K"])
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

Accuracy: 0.7997
Precision: 0.5963
Recall: 0.6316
F1: 0.6134
